## 安装库  Installation must library

In [ ]:
!pip install timm
!pip install yacs
!pip install pandas
!pip install scipy
!pip install termcolor

In [ ]:
!pip install pytorch_metric_learning

In [11]:
!git clone https://github.com/NVIDIA/apex

fatal: destination path 'apex' already exists and is not an empty directory.


In [ ]:
cd /root/autodl-tmp/main/apex

In [ ]:
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

## 先划分验证集放到data下去，注意row的列数6,7运行错误基本是这个问题   
## You can choose to repartition the dataset, or you can skip it and change the csv name

In [ ]:
cd /root/autodl-tmp/main/data/snake

In [ ]:
!python split_snake.py

## 生成混合数据json文件放到data下,注意row的列数6,7运行错误基本是这个问题 
## get meta information

In [ ]:
!python get_meta_data.py

## 清除csv里边没有路径的行
## Clear images without path in csv

In [ ]:
cd /root/autodl-tmp/data/

In [20]:
import csv
import os
data_root = "/root/autodl-tmp/data/SnakeCLEF2023-small_size/"
with open('valid_split.csv', 'r') as file:
    reader = csv.DictReader(file)

    # 创建新的csv文件并写入标题行
    with open('val.csv', 'w', newline='') as new_file:
        writer = csv.DictWriter(new_file, fieldnames=reader.fieldnames)
        writer.writeheader()

        # 遍历每一行
        for row in reader:
            path = os.path.join(data_root, row['image_path'])
            # 如果image_path存在，则写入新文件
            if os.path.exists(path):
                writer.writerow(row)
with open('train_split.csv', 'r') as file:
    reader = csv.DictReader(file)

    # 创建新的csv文件并写入标题行
    with open('train.csv', 'w', newline='') as new_file:
        writer = csv.DictWriter(new_file, fieldnames=reader.fieldnames)
        writer.writeheader()

        # 遍历每一行
        for row in reader:
            path = os.path.join(data_root, row['image_path'])
            # 如果image_path存在，则写入新文件
            if os.path.exists(path):
                writer.writerow(row)

## Train    种类数变为1784

In [ ]:
cd /root/autodl-tmp/main

In [ ]:
!python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345  main_CallArcloss.py --cfg ./configs/MetaFG_meta_2_384_snake.yaml --batch-size 22 --tag OUTPUT_TAG --lr 5e-5  --min-lr 5e-7 --warmup-lr 5e-8 --epochs 100 --warmup-epochs 20 --dataset snakeclef2023  --opts DATA.IMG_SIZE 384 TRAIN.AUTO_RESUME False --output output1  --amp-opt-level O1 --pretrain /root/autodl-tmp/main/metafg_2_inat21_384.pth

In [ ]:
!python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345  main_simclr.py --cfg ./configs/MetaFG_meta_2_384_snake.yaml --batch-size 10 --tag OUTPUT_TAG --lr 5e-4  --min-lr 5e-7 --warmup-lr 5e-8 --epochs 100 --warmup-epochs 20 --dataset snakeclef2023  --opts DATA.IMG_SIZE 384 TRAIN.AUTO_RESUME False --output output5  --amp-opt-level O1 --pretrain /root/autodl-tmp/main/metafg_2_inat21_384.pth

## Test

In [ ]:
!python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345  main_test.py --eval --cfg ./configs/MetaFG_meta_2_384_snake.yaml --batch-size 16 --tag OUTPUT_TAG --dataset snakeclef2023test --resume /root/autodl-tmp/main/ckpt_epoch_99.pth --opts DATA.IMG_SIZE 384 TRAIN.AUTO_RESUME False

In [ ]:
#生成去除重复后的casv文件
# select the class with the most occurrences from the csv files generated by the three models as the final prediction result.
!python post_process.py

## csv融合
## Integration of three csv result files

In [ ]:
!python merge_multiple.py